# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [19]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
import nltk
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from io import StringIO

Загружаем отобранные статьи 

In [20]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(StringIO(read_data), orient='records')

Формируем предложения из статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложени и их длину

In [21]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"sentence": [], "len": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [s, len(s)]
print("Number of sentences: ", len(sentences.index))
sentences.head(3)

Number of sentences:  222


,sentence,len
0,В Госдуме предложили сделать старый Новый год ...,84
1,Председатель союза дачников Подмосковья и депу...,234
2,"""После длинных новогодних праздников людям тяж...",176


In [22]:
train_text = sentences['sentence'].astype('str')

Задание всех seed

In [23]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [25]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /cointegrated/rubert-tiny2/26ebb6db2a68593c54c74902d7a74f332da66297693f965cc9f1b0af4abf3894?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1705589028&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTU4OTAyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9jb2ludGVncmF0ZWQvcnViZXJ0LXRpbnkyLzI2ZWJiNmRiMmE2ODU5M2M1NGM3NDkwMmQ3YTc0ZjMzMmRhNjYyOTc2OTNmOTY1Y2M5ZjFiMGFmNGFiZjM4OTQ~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=EDz5Mdv1dmoh4tqCM11IClloUdT4Br8WJzPKfstPRhj8zyVJ0rAlnDrPaGKOskEej4aQ~cfE7Ih0tZ3gOIucz2QlJ9jLl69hVNZZamWatkiaHsUBCivF1Gy2BwMoScJaGYKV7maHYSgTbnMUqpGsOVnXuNIsWB1TybC1xe14ecGFy9Fcy6WvZtfTEjv9j0wUXbPg~9DJQpvpLK5noiJoRFs28mT0fA4jaIDvfIynkVIECvQC60tKC8cS8PmA6Ua~rPzXSOB7iyVZwhc7~peBsZ92dN4cqj6aISZmQijbto4Jm~l

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    padding = True,
    truncation = True
)

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train)